In [1]:
# import Libraries

import cohere
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains.question_answering import load_qa_chain



In [1]:
%pip install \
    "pinecone[grpc]" \
    "langchain-pinecone" \
    "langchain-openai" \
    "langchain-text-splitters" \
    "langchain"

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
# ## Lets Read the document
# def read_doc(directory):
#     file_loader=PyPDFDirectoryLoader(directory)
#     documents=file_loader.load(directory)
#     return documents

In [3]:
from langchain.document_loaders import PyPDFLoader

def read_doc(file_path):
    file_loader = PyPDFLoader(file_path)
    document = file_loader.load()
    return document

# Load the single PDF
doc = read_doc("Black Holes Explained.pdf")

# Check the length of the document
len(doc)


92

In [4]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc
type(doc)

list

In [5]:
api_key="a5ed3c1e-9541-431b-9b10-5d6ae6876eb9"

In [6]:
from pinecone.grpc import PineconeGRPC as Pinecone
import os
from pinecone import Pinecone,ServerlessSpec
index_name="rag-getting-started"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [13]:
index=pc.Index(index_name)
print(index)

from langchain_pinecone import PineconeVectorStore


#index_name = "pdf-qa-index"
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)


#index_name = "langchain-test-index"




In [14]:
import cohere

# Initialize Cohere client
co = cohere.Client('VFeNivpxvzvHHyWz8l5QcFf3FIl5vKuCIV1GS5kg')  # Replace with your Cohere API key

# Define the texts to embed
texts = ["Black Holes Explained.pdf"]

# Generate embeddings
embeddings = [co.embed(texts=texts).embeddings]

print(embeddings)


[[[-0.9501953, -0.39453125, 2.2578125, -1.6572266, -1.0136719, -0.67089844, -0.4230957, -1.0537109, 2.1035156, 0.91015625, -0.34814453, 0.8208008, -3.5273438, -0.13439941, -0.03781128, -1.4941406, 1.25, 1.0478516, 1.7392578, -0.19335938, -0.91845703, 0.9736328, 2.5722656, -1.5087891, 0.9604492, 1.0722656, 2.5195312, -1.6982422, -0.47387695, -0.08215332, -0.33544922, 3.2246094, 0.78759766, 0.22631836, 1.8457031, 0.28344727, 1.8154297, 0.61328125, 2.4160156, -0.20397949, 0.5678711, 0.5854492, -1.2509766, 0.6274414, 0.6123047, 0.46435547, 0.86816406, 1.6025391, 0.06567383, -0.015701294, -0.6850586, 0.23596191, -0.70410156, 1.8427734, 1.9394531, -0.019882202, -3.3691406, -0.84375, -0.55029297, 0.052215576, -0.59716797, 1.1054688, -0.20922852, 0.46044922, 0.3881836, -1.0546875, -1.0341797, 0.42138672, -0.94628906, -1.2841797, 1.0849609, 1.1855469, -3.1621094, -1.0107422, -0.2770996, -1.3818359, -1.453125, -2.6054688, 0.22937012, 0.5708008, 1.7441406, -0.0725708, -2.2070312, 0.22131348, -2.0

In [17]:
from langchain_pinecone import PineconeEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import MarkdownHeaderTextSplitter
import os
import time

# Chunk the document based on h2 headers.
markdown_document = "## Introduction\n\nWelcome to the whimsical world of the WonderVector5000, an astonishing leap into the realms of imaginative technology. This extraordinary device, borne of creative fancy, promises to revolutionize absolutely nothing while dazzling you with its fantastical features. Whether you're a seasoned technophile or just someone looking for a bit of fun, the WonderVector5000 is sure to leave you amused and bemused in equal measure. Let's explore the incredible, albeit entirely fictitious, specifications, setup process, and troubleshooting tips for this marvel of modern nonsense.\n\n## Product overview\n\nThe WonderVector5000 is packed with features that defy logic and physics, each designed to sound impressive while maintaining a delightful air of absurdity:\n\n- Quantum Flibberflabber Engine: The heart of the WonderVector5000, this engine operates on principles of quantum flibberflabber, a phenomenon as mysterious as it is meaningless. It's said to harness the power of improbability to function seamlessly across multiple dimensions.\n\n- Hyperbolic Singularity Matrix: This component compresses infinite possibilities into a singular hyperbolic state, allowing the device to predict outcomes with 0% accuracy, ensuring every use is a new adventure.\n\n- Aetherial Flux Capacitor: Drawing energy from the fictional aether, this flux capacitor provides unlimited power by tapping into the boundless reserves of imaginary energy fields.\n\n- Multi-Dimensional Holo-Interface: Interact with the WonderVector5000 through its holographic interface that projects controls and information in three-and-a-half dimensions, creating a user experience that's simultaneously futuristic and perplexing.\n\n- Neural Fandango Synchronizer: This advanced feature connects directly to the user's brain waves, converting your deepest thoughts into tangible actions—albeit with results that are whimsically unpredictable.\n\n- Chrono-Distortion Field: Manipulate time itself with the WonderVector5000's chrono-distortion field, allowing you to experience moments before they occur or revisit them in a state of temporal flux.\n\n## Use cases\n\nWhile the WonderVector5000 is fundamentally a device of fiction and fun, let's imagine some scenarios where it could hypothetically be applied:\n\n- Time Travel Adventures: Use the Chrono-Distortion Field to visit key moments in history or glimpse into the future. While actual temporal manipulation is impossible, the mere idea sparks endless storytelling possibilities.\n\n- Interdimensional Gaming: Engage with the Multi-Dimensional Holo-Interface for immersive, out-of-this-world gaming experiences. Imagine games that adapt to your thoughts via the Neural Fandango Synchronizer, creating a unique and ever-changing environment.\n\n- Infinite Creativity: Harness the Hyperbolic Singularity Matrix for brainstorming sessions. By compressing infinite possibilities into hyperbolic states, it could theoretically help unlock unprecedented creative ideas.\n\n- Energy Experiments: Explore the concept of limitless power with the Aetherial Flux Capacitor. Though purely fictional, the notion of drawing energy from the aether could inspire innovative thinking in energy research.\n\n## Getting started\n\nSetting up your WonderVector5000 is both simple and absurdly intricate. Follow these steps to unleash the full potential of your new device:\n\n1. Unpack the Device: Remove the WonderVector5000 from its anti-gravitational packaging, ensuring to handle with care to avoid disturbing the delicate balance of its components.\n\n2. Initiate the Quantum Flibberflabber Engine: Locate the translucent lever marked “QFE Start” and pull it gently. You should notice a slight shimmer in the air as the engine engages, indicating that quantum flibberflabber is in effect.\n\n3. Calibrate the Hyperbolic Singularity Matrix: Turn the dials labeled 'Infinity A' and 'Infinity B' until the matrix stabilizes. You'll know it's calibrated correctly when the display shows a single, stable “∞”.\n\n4. Engage the Aetherial Flux Capacitor: Insert the EtherKey into the designated slot and turn it clockwise. A faint humming sound should confirm that the aetherial flux capacitor is active.\n\n5. Activate the Multi-Dimensional Holo-Interface: Press the button resembling a floating question mark to activate the holo-interface. The controls should materialize before your eyes, slightly out of phase with reality.\n\n6. Synchronize the Neural Fandango Synchronizer: Place the neural headband on your forehead and think of the word “Wonder”. The device will sync with your thoughts, a process that should take just a few moments.\n\n7. Set the Chrono-Distortion Field: Use the temporal sliders to adjust the time settings. Recommended presets include “Past”, “Present”, and “Future”, though feel free to explore other, more abstract temporal states.\n\n## Troubleshooting\n\nEven a device as fantastically designed as the WonderVector5000 can encounter problems. Here are some common issues and their solutions:\n\n- Issue: The Quantum Flibberflabber Engine won't start.\n\n    - Solution: Ensure the anti-gravitational packaging has been completely removed. Check for any residual shards of improbability that might be obstructing the engine.\n\n- Issue: The Hyperbolic Singularity Matrix displays “∞∞”.\n\n    - Solution: This indicates a hyper-infinite loop. Reset the dials to zero and then adjust them slowly until the display shows a single, stable infinity symbol.\n\n- Issue: The Aetherial Flux Capacitor isn't engaging.\n\n    - Solution: Verify that the EtherKey is properly inserted and genuine. Counterfeit EtherKeys can often cause malfunctions. Replace with an authenticated EtherKey if necessary.\n\n- Issue: The Multi-Dimensional Holo-Interface shows garbled projections.\n\n    - Solution: Realign the temporal resonators by tapping the holographic screen three times in quick succession. This should stabilize the projections.\n\n- Issue: The Neural Fandango Synchronizer causes headaches.\n\n    - Solution: Ensure the headband is properly positioned and not too tight. Relax and focus on simple, calming thoughts to ease the synchronization process.\n\n- Issue: The Chrono-Distortion Field is stuck in the past.\n\n    - Solution: Increase the temporal flux by 5%. If this fails, perform a hard reset by holding down the “Future” slider for ten seconds."

headers_to_split_on = [
    ("##", "Header 2")
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

# Initialize a LangChain embedding object.
model_name = "multilingual-e5-large"  
embeddings = PineconeEmbeddings(  
    model=model_name,  
    pinecone_api_key=os.environ.get("PINECONE_API_KEY")  
)  

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=md_header_splits,
    index_name="rag-getting-started",
    embedding=embeddings, 
    namespace="wondervector5000" 
)

time.sleep(1)


In [18]:
index = pc.Index(index_name)
namespace = "wondervector5000"

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0], 
        namespace=namespace, 
        top_k=3,
        include_values=True,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '1b50b490-1de3-412f-8907-130328f83f94',
              'metadata': {'Header 2': 'Use cases',
                           'text': '## Use cases  \n'
                                   'While the WonderVector5000 is '
                                   'fundamentally a device of fiction and fun, '
                                   "let's imagine some scenarios where it "
                                   'could hypothetically be applied:  \n'
                                   '- Time Travel Adventures: Use the '
                                   'Chrono-Distortion Field to visit key '
                                   'moments in history or glimpse into the '
                                   'future. While actual temporal manipulation '
                                   'is impossible, the mere idea sparks '
                                   'endless storytelling possibilities.  \n'
                                   '- Interdimensional Gaming: Engage with the 

In [20]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

In [21]:
# Initialize the LLM with Llama 3.1 model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

In [22]:
from langchain.chains import RetrievalQA  
# Initialize a LangChain object for chatting with the LLM
# with knowledge from Pinecone. 
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [ ]:
# Define a few questions about the WonderVector5000.
query1 = """What are the first 3 steps for getting started 
with the WonderVector5000?"""

query2 = """The Neural Fandango Synchronizer is giving me a 
headache. What do I do?"""

# Send each query to the LLM twice, first with relevant knowledge from Pincone 
# and then without any additional knowledge.
print("Query 1\n")
print("Chat with knowledge:")
print(qa.invoke(query1).get("result"))
print("\nChat without knowledge:")
print(llm.invoke(query1).content)
print("\nQuery 2\n")
print("Chat with knowledge:")
print(qa.invoke(query2).get("result"))
print("\nChat without knowledge:")
print(llm.invoke(query2).content)
